# Load Image

In [11]:
def load_img(f):
    f=open(f)
    lines=f.readlines()
    imgs, lab=[], []
    for i in range(len(lines)):
        fn, label = lines[i].split(' ')
        
        
    imgs= np.asarray(imgs, np.float32)
    lab= np.asarray(lab, np.int32)
    return imgs, lab 
f1 = open('train.txt')
lines=f1.readlines()
lines
f1.close()
tr_lab=[]
tr_filename=[]
for i in range(len(lines)):
    fn, label = lines[i].split(' ')
    tr_filename.append(fn)
    tr_lab.append(label)
for i in range(len(tr_lab)):
    tr_lab[i]=tr_lab[i].replace("\n", "")

In [13]:
unique_labels = set(tr_lab)
len(unique_labels)

50

In [14]:
f2 = open('test.txt')
lines=f2.readlines()
#lines
f2.close()
te_lab=[]
te_filename=[]
for i in range(len(lines)):
    fn, label = lines[i].split(' ')
    te_filename.append(fn)
    te_lab.append(label)
    for i in range(len(te_lab)):
        te_lab[i]=te_lab[i].replace("\n", "")

In [15]:
f3 = open('val.txt')
lines=f3.readlines()
#lines
f3.close()
va_lab=[]
va_filename=[]
for i in range(len(lines)):
    fn, label = lines[i].split(' ')
    va_filename.append(fn)
    va_lab.append(label)
    for i in range(len(va_lab)):
        va_lab[i]=va_lab[i].replace("\n", "")

# Task: Designing a Convolution Module for Variable Input Channels

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class CustomConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(CustomConv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.randn(out_channels))

    def forward(self, x):
        if self.padding > 0:
            x = F.pad(x, (self.padding, self.padding, self.padding, self.padding))

        out = F.conv2d(x, self.weight, bias=self.bias, stride=self.stride, padding=0)
        return out

class CNN(nn.Module):
    def __init__(self, unique_labels):
        super(CNN, self).__init__()
        self.conv1 = CustomConv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, len(unique_labels))  # 使用 len(unique_labels) 作为输出类别数

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = CNN(unique_labels)
optimizer = optim.Adam(model.parameters(), lr=0.001)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [37]:
import os
import numpy as np
from PIL import Image

def load_images_from_folder(filenames):
    images = []
    for filename in filenames:
        if filename.endswith(".png") or filename.endswith(".JPEG"):
            img = Image.open(filename).convert('L')
            img = img.resize((28, 28))
            img_array = np.array(img)
            images.append(img_array)
    return np.array(images)

In [19]:
train_images = load_images_from_folder(tr_filename)

test_images = load_images_from_folder(te_filename)
val_images = load_images_from_folder(va_filename)

In [21]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor([int(label) for label in labels], dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [22]:
train_images = train_images.reshape(train_images.shape[0], 1, 28, 28) / 255.0
test_images = test_images.reshape(test_images.shape[0], 1, 28, 28) / 255.0
val_images = val_images.reshape(val_images.shape[0], 1, 28, 28) / 255.0

In [23]:
train_dataset = MyDataset(train_images, tr_lab)
test_dataset = MyDataset(test_images, te_lab)
val_dataset = MyDataset(val_images, va_lab)

In [8]:
len(va_lab)

450

In [24]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=64, shuffle=True)

### Model Training

In [41]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
n_epochs = 10

for epoch in range(n_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 3.900797128677368
Epoch 2/10, Loss: 3.9015893936157227
Epoch 3/10, Loss: 3.90079927444458
Epoch 4/10, Loss: 3.899658441543579
Epoch 5/10, Loss: 3.9002490043640137
Epoch 6/10, Loss: 3.912843942642212
Epoch 7/10, Loss: 3.9088165760040283
Epoch 8/10, Loss: 3.9270849227905273
Epoch 9/10, Loss: 3.9512574672698975
Epoch 10/10, Loss: 3.8994576930999756


In [42]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f'Test accuracy: {accuracy * 100:.2f}%')

Test accuracy: 2.00%


In [43]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in val_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f'Test accuracy: {accuracy * 100:.2f}%')

Test accuracy: 2.00%


## Designing a Two-Layer Network for Image Classification

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.models import resnet34
from torch.utils.data import DataLoader


### ResNet34

In [15]:
model = resnet34(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

num_classes = len(unique_labels)
model.fc = nn.Linear(model.fc.in_features, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    
    # 验证模型
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')

print('Finished Training')


C:\Users\andre\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\andre\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/10], Loss: 3.6808
Validation Loss: 3.5128, Accuracy: 9.33%
Epoch [2/10], Loss: 3.4841
Validation Loss: 3.4823, Accuracy: 9.11%
Epoch [3/10], Loss: 3.3415
Validation Loss: 4.0235, Accuracy: 9.11%
Epoch [4/10], Loss: 3.2232
Validation Loss: 4.7228, Accuracy: 8.44%
Epoch [5/10], Loss: 3.1211
Validation Loss: 3.3992, Accuracy: 17.11%
Epoch [6/10], Loss: 3.0167
Validation Loss: 3.1358, Accuracy: 13.33%
Epoch [7/10], Loss: 2.9022
Validation Loss: 3.2252, Accuracy: 20.89%
Epoch [8/10], Loss: 2.7838
Validation Loss: 2.9911, Accuracy: 19.56%
Epoch [9/10], Loss: 2.6572
Validation Loss: 2.9629, Accuracy: 20.67%
Epoch [10/10], Loss: 2.5106
Validation Loss: 3.2035, Accuracy: 16.22%
Finished Training


In [17]:
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {100 * correct / total:.2f}%')

Test Loss: 3.5081, Test Accuracy: 14.44%


## Custer

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# 定义残差块
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out
class CusCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CusCNN, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(64, 3)
        self.layer2 = self._make_layer(128, 4, stride=2)
        self.layer3 = self._make_layer(256, 6, stride=2)
        #self.layer4 = self._make_layer(512, 3, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [33]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # 灰度图像只有一个通道
])

In [34]:
model = CusCNN(num_classes=len(unique_labels))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')

print('Finished Training')

Epoch [1/10], Loss: 3.6193
Validation Loss: 3.4426, Accuracy: 9.56%
Epoch [2/10], Loss: 3.3462
Validation Loss: 4.2975, Accuracy: 9.11%
Epoch [3/10], Loss: 3.1672
Validation Loss: 3.3874, Accuracy: 15.78%
Epoch [4/10], Loss: 3.0356
Validation Loss: 3.6999, Accuracy: 14.00%
Epoch [5/10], Loss: 2.9067
Validation Loss: 3.6091, Accuracy: 8.44%
Epoch [6/10], Loss: 2.7771
Validation Loss: 3.0082, Accuracy: 18.00%
Epoch [7/10], Loss: 2.6253
Validation Loss: 2.9606, Accuracy: 21.11%
Epoch [8/10], Loss: 2.4605
Validation Loss: 3.4873, Accuracy: 16.67%
Epoch [9/10], Loss: 2.2625
Validation Loss: 3.0924, Accuracy: 23.11%
Epoch [10/10], Loss: 2.0180
Validation Loss: 3.2424, Accuracy: 18.89%
Finished Training


In [35]:
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {100 * correct / total:.2f}%')

Test Loss: 3.3159, Test Accuracy: 21.11%
